# This notebook add all the data to the same "master" dataset, including documentation of the variables and all the years.

1. Download documentation for the variables.
2. Select the variables needed based on the paper.
3. Merge all the files together


In [69]:
import os
#%load_ext cudf.pandas
import pandas as pd
import numpy as np
from glob import glob
import requests
from bs4 import BeautifulSoup
import io
import pyreadstat

In [70]:
DATA_PATH = "/Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/"

# 1. Download NHANES Data

Downloading the files takess from 15 min to 30 min per year with high speed internet, the good thing is that in can get interrupted at it skips the files already downloaded - just make sure the last file was correctly writen. 

In [72]:
def scrape_nhanes_xpt_files(year, DATA_PAT=DATA_PATH):
    """
    Note PAXMIN.XPT aka "Physical Activity Monitor - Minute	" is the only file missing.
    It's +6 gigas and CDC website its not preciselly fast.
    It takes 6 hours to download usually.
    Polling data without a unique identifer also will be missing ("*POL*.parquet") since
    I have no use for it
    """

    list_types = ["Demographics", "Dietary", "Examination", "Laboratory", "Questionnaire"]

    # Create folder structure for the data based on the year
    os.chdir(DATA_PATH)
    os.makedirs(f"{year}-{year+1}", exist_ok=True)
    os.chdir(f"{year}-{year+1}")

    print(f"NHANES Data from {year}-{year+1} year")
    print("__________________________")
    print("__________________________")

    for type in list_types:
        # Type of data and year
        url = f"https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component={type}&CycleBeginYear={year}"

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            continue

        # Create folder structure for the data based on the data type
        os.makedirs(type, exist_ok=True)
        os.chdir(type)
        print("### Data type:", type, "###")
        print("__________________________")

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all links that end with .XPT (case insensitive)
        xpt_links = soup.find_all('a', href=lambda href: href and href.lower().endswith('.xpt'))

        # Download and process each XPT file
        for link in xpt_links:
            file_url = link['href']
            if not file_url.startswith('http') and not "PAXMIN" in file_url and not "POL" in file_url:
                file_url = f"https://wwwn.cdc.gov{file_url}"
            else:
                continue

            file_name = file_url.split('/')[-1]
            parquet_filename = file_name.replace('.XPT', '.parquet')

            # Download the XPT file if it doesn't exist
            if not os.path.exists(parquet_filename):
                print(f"Downloading {file_name} from CDC website...")
                file_response = requests.get(file_url)
                if file_response.status_code == 200:
                    # Create a temporary file so pyreadstat can read it
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.XPT') as temp_file:
                        temp_file.write(file_response.content)
                        temp_file_path = temp_file.name

                    xpt_data, _ = pyreadstat.read_xport(temp_file_path, encoding='cp1252')
                    xpt_data.to_parquet(parquet_filename, index=False)

                    # Remove the temporary file
                    os.unlink(temp_file_path)
                    print(f"Saved as {parquet_filename}")
                else:
                    print(f"Failed to download {file_name}. Status code: {file_response.status_code}")
            else:
                print(f"{parquet_filename} file already in the destination folder")

        # Moving again to parent directory
        os.chdir('..')
        print("__________________________")

years = [1999,2001,2003,2005,2007,2009,2011,2013]
for year in years:
    scrape_nhanes_xpt_files(year)


NHANES Data from 1999-2000 year
__________________________
__________________________
### Data type: Demographics ###
__________________________
DEMO.parquet file already in the destination folder
__________________________
### Data type: Dietary ###
__________________________
DRXIFF.parquet file already in the destination folder
DRXTOT.parquet file already in the destination folder
DRXFMT.parquet file already in the destination folder
DSBI.parquet file already in the destination folder
DSII.parquet file already in the destination folder
DSPI.parquet file already in the destination folder
DSQFILE1.parquet file already in the destination folder
DSQFILE2.parquet file already in the destination folder
__________________________
### Data type: Examination ###
__________________________
AUX1.parquet file already in the destination folder
AUXAR.parquet file already in the destination folder
AUXTYM.parquet file already in the destination folder
BAX.parquet file already in the destination fold

# 2. Download documentation for the variables.

In [73]:
def create_document_variables():

    df_docs = pd.DataFrame()
    data_type = ["Demographics", "Dietary", "Examination", "Questionnaire", "Laboratory"]
    for type in data_type:
        try:
            url = f"https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component={type}&Cycle="
            df = pd.read_html(url)[0]
            df_docs = pd.concat([df_docs, df], ignore_index=True)
            print(f"Successfully scraped: {type} variable documentation")
        except Exception as e:
            print(f"Error scraping {type}: {str(e)}")

    df_docs['Year'] = df_docs['Begin Year'].astype(str) + "-" + df_docs['EndYear'].astype(str)
    
    return df_docs


#docs = create_document_variables()
#docs.to_csv(DATA_PATH + "documentation_variables.csv",index=False)
docs = pd.read_csv(DATA_PATH + "documentation_variables.csv")
docs

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
0,AIALANG,Language of the MEC ACASI Interview Instrument,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
1,DMDBORN,In what country {were you/was SP} born?,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
2,DMDCITZN,{Are you/Is SP} a citizen of the United States...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
3,DMDEDUC2,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
4,DMDEDUC3,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
...,...,...,...,...,...,...,...,...,...
60957,LBXPFHS,Perfluorohexane sulfonic acid (PFHxS) (ng/mL),P_PFAS,Perfluoroalkyl and Polyfluoroalkyl Substances,2017,2020,Laboratory,NaN,2017-2020
60958,LBXPFNA,Perfluorononanoic acid (PFNA) (ng/mL),P_PFAS,Perfluoroalkyl and Polyfluoroalkyl Substances,2017,2020,Laboratory,NaN,2017-2020
60959,LBXPFUA,Perfluoroundecanoic acid (PFUA) (ng/mL),P_PFAS,Perfluoroalkyl and Polyfluoroalkyl Substances,2017,2020,Laboratory,NaN,2017-2020
60960,SEQN,Respondent sequence number.,P_PFAS,Perfluoroalkyl and Polyfluoroalkyl Substances,2017,2020,Laboratory,NaN,2017-2020


In [74]:
def glance_var_docs(var):
    df = docs[docs["Variable Name"] == var].sort_values("Year")
    df = df[df["Use Constraints"] != "RDC Only"]
    return df

glance_var_docs("RIDRETH1")


,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
195,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO,Demographic Variables & Sample Weights,1999,2000,Demographics,NaN,1999-2000
156,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO_B,Demographic Variables & Sample Weights,2001,2002,Demographics,NaN,2001-2002
118,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO_C,Demographic Variables & Sample Weights,2003,2004,Demographics,NaN,2003-2004
32,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,NaN,2005-2006
75,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO_E,Demographic Variables & Sample Weights,2007,2008,Demographics,NaN,2007-2008
55991,RIDRETH1,Recode of reported race and Hispanic origin in...,PSTPOL_F,Pesticides - Organochlorine Pesticides - Poole...,2009,2010,Laboratory,NaN,2009-2010
55753,RIDRETH1,Ethnicity - Recode,DOXPOL_F,"Polychlorinated dibenzo-p-dioxins (PCDDs), Dib...",2009,2010,Laboratory,NaN,2009-2010
56111,RIDRETH1,Recode of reported race and Hispanic origin in...,PCBPOL_F,Non-dioxin-like Polychlorinated Biphenyls & Mo...,2009,2010,Laboratory,NaN,2009-2010
343,RIDRETH1,Recode of reported race and ethnicity informat...,DEMO_F,Demographic Variables & Sample Weights,2009,2010,Demographics,NaN,2009-2010
56033,RIDRETH1,Recode of reported race and Hispanic origin in...,BFRPOL_F,Brominated Flame Retardants (BFRs) - Pooled Sa...,2009,2010,Laboratory,NaN,2009-2010


# 3. Compile the variables needed based on paper.

There are more than 3000 variables/columns in NHANES, we'll compile/stack only the ones we need to replicate the papers


In [75]:
def compile_data(variable_list, DATA_PATH=DATA_PATH, save_file_as=False):
    """
    for var in variable_list that you want: 
        - Look in the docs what parquet files contain that variable
        - Make a list called parquet_files that contains the path of the files.
        for file in parquet_files:
            - Read that file specific column variable + SEQN
            - Concat to the "master" dataframe 
            - Save the file if choosen 

    """
    docs_df = pd.read_csv(DATA_PATH + "documentation_variables.csv")
    docs_df = docs_df[docs_df["Use Constraints"] != "RDC Only"]

    # Initial dataset just with all the individual indexes and its year
    master_df = pd.DataFrame()

    file_path = sorted(glob(DATA_PATH + "**" + "/*DEMO*.parquet", recursive=True))
    for file in file_path:
        df = pd.read_parquet(file, columns=["SEQN"])
        df["YEAR"] = file[55:64]
        master_df = pd.concat([master_df, df], ignore_index=True)
    master_df.sort_values(by=["SEQN", "YEAR"], inplace=True)


    for var in variable_list:
        print(f"Searching for variable {var} ...")
        var = var.upper()
        parquet_files = sorted(docs_df[docs_df["Variable Name"] == var]['Data File Name'].unique())

        variable_concat_df = pd.DataFrame()
        for file in parquet_files:
            pattern = os.path.join(DATA_PATH, "**", f"{file}.parquet")
            file_path = glob(pattern, recursive=True)
            file_path = ''.join(file_path)
            if file_path:
                df = pd.read_parquet(file_path, columns=["SEQN", var])
                variable_concat_df = pd.concat([variable_concat_df, df], ignore_index=True)
                print(f"--> Successfully added: {var} from {file_path}")

        master_df = master_df.merge(variable_concat_df, on = ["SEQN"], how="left")

    if save_file_as:
        master_df.to_parquet(DATA_PATH + f"{save_file_as}.parquet", index=False)
        print("File saved inthe following folder: ", DATA_PATH)
    
    
    return master_df

## 3.1 Dinh et al. (2019)

The following file was created manually searching for the varaible names in the NHANES online "variable search tool" and taking the variables from the paper's figures.

In [76]:
dinh_2019_vars = pd.read_excel(DATA_PATH + "dinh_2019_variables_doc.xlsx")
dinh_2019_vars

,Variable Name,NHANES Name,NHANES File,NHANES Type of data,Variable Definition
0,Age,RIDAGEYR,DEMO,Demographics,Best age in years of the sample person at time...
1,Alcohol consumption,ALQ130,ALQ,Questionnaire,"In the past 12 months, on those days that {you..."
2,Alcohol intake,DRXTALCO,DRXTOT,Dietary,Alcohol (gm) - Total Nutrient Intakes
3,"Alcohol intake, First Day",DR1TALCO,DR1TOT,Dietary,"Alcohol (gm) - Total Nutrient Intakes, First Day"
4,"Alcohol intake, Second Day",DR2TALCO,DR2TOT,Dietary,"Alcohol (gm) - Total Nutrient Intakes, Second ..."
...,...,...,...,...,...
57,Triglyceride,LBDSTRSI,"LAB18, L40, BIOPRO",Laboratory,Triglycerides (mmol/L)
58,Waist circumference,BMXWAIST,BMX,Examination,Waist Circumference (cm)
59,Weight,BMXWT,BMX,Examination,Weight (kg)
60,White blood cell count,LBXWBCSI,"LAB25, L25, CBC",Laboratory,White blood cell count (1000 cells/uL)


In [77]:
dinh_variables = dinh_2019_vars["NHANES Name"].unique()

df = compile_data(dinh_variables, save_file_as="dinh_raw_data")

Searching for variable RIDAGEYR ...
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/1999-2000/Demographics/DEMO.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2001-2002/Demographics/DEMO_B.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2003-2004/Demographics/DEMO_C.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2005-2006/Demographics/DEMO_D.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2007-2008/Demographics/DEMO_E.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2009-2010/Demographics/DEMO_F.parquet
--> Successfully added: RIDAGEYR from /Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/2011-2012/Demographics/DEMO_G.parquet
--> Successfully added: RIDAGEYR from /Users/pipe

In [78]:
df

,SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY1,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI
0,1.0,1999-2000,2.0,NaN,0.00,NaN,NaN,15.2,18.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,45.7,12.5,NaN,NaN
1,2.0,1999-2000,77.0,1.0,0.00,NaN,NaN,29.8,38.2,288.0,...,106.0,NaN,98.0,98.0,5.56,1.298,98.0,75.4,7.6,19.0
2,3.0,1999-2000,10.0,NaN,0.00,NaN,NaN,19.7,25.5,NaN,...,110.0,NaN,104.0,112.0,3.34,NaN,64.7,32.9,7.5,NaN
3,4.0,1999-2000,1.0,NaN,0.00,NaN,NaN,16.4,20.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.3,8.8,NaN
4,5.0,1999-2000,49.0,3.0,34.56,NaN,NaN,35.8,39.7,276.0,...,122.0,NaN,122.0,122.0,7.21,3.850,99.9,92.5,5.9,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82086,83727.0,2013-2014,26.0,3.0,NaN,14.0,19.9,29.9,35.2,285.0,...,110.0,NaN,116.0,112.0,4.91,0.858,87.1,71.8,5.1,27.0
82087,83728.0,2013-2014,2.0,NaN,NaN,0.0,0.0,14.7,16.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,47.2,11.3,6.6,NaN
82088,83729.0,2013-2014,42.0,NaN,NaN,0.0,0.0,37.0,37.6,277.0,...,136.0,NaN,130.0,138.0,3.93,1.197,102.7,89.6,6.4,26.0
82089,83730.0,2013-2014,7.0,NaN,NaN,NaN,NaN,19.0,26.0,NaN,...,NaN,NaN,NaN,NaN,4.32,NaN,53.0,22.8,9.9,NaN


# 4. Prep the data

There are some fixes before the data is ready for analysis from variables that are the same but called diffrenet in different NHANES years:

1. Intake variables went from 1 day in 1999 to 2001 to 2 days from 2003 on, therefore the variable has to be homogenized. I take the average of both days. This happends with:

- Alcohol intake (`DRXTALCO`, `DR1TALCO`, `DR2TALCO`)
- Caffeine intake (`DRXTCAFF`, `DR1TCAFF`, `DR2TCAFF`)
- Calcium intake (`DRXTCALC`, `DR1TCALC`, `DR2TCALC`)
- Carbohydrate intake (`DRXTCARB`, `DR1TCARB`, `DR2TCARB`)
- Fiber intake (`DRXTFIBE`, `DR1TFIBE`, `DR2TFIBE`)
- Kcal intake (`DRXTKCAL`, `DR1TKCAL`, `DR2TKCAL`)
- Sodium intake (`DRDTSODI`, `DR1TSODI`, `DR2TSODI`)


In [522]:
glance_var_docs("DRDTSODI")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
1889,DRDTSODI,Sodium (mg) (adjusted for salt use in food pre...,DRXTOT,Dietary Interview - Total Nutrient Intakes,1999,2000,Dietary,NaN,1999-2000
2058,DRDTSODI,Sodium (mg) (adjusted for salt use in food pre...,DRXTOT_B,Dietary Interview - Total Nutrient Intakes,2001,2002,Dietary,NaN,2001-2002


In [523]:
glance_var_docs("DR1TSODI")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
2738,DR1TSODI,Sodium (mg) (adjusted for salt use in food pre...,DR1TOT_C,"Dietary Interview - Total Nutrient Intakes, Fi...",2003,2004,Dietary,NaN,2003-2004
5542,DR1TSODI,Sodium (mg) (adjusted for salt use in food pre...,DR1TOT_D,"Dietary Interview - Total Nutrient Intakes, Fi...",2005,2006,Dietary,NaN,2005-2006
4416,DR1TSODI,Sodium (mg) (adjusted for salt use in food pre...,DR1TOT_E,"Dietary Interview - Total Nutrient Intakes, Fi...",2007,2008,Dietary,NaN,2007-2008
3684,DR1TSODI,Sodium (mg),DR1TOT_F,"Dietary Interview - Total Nutrient Intakes, Fi...",2009,2010,Dietary,NaN,2009-2010
4713,DR1TSODI,Sodium (mg),DR1TOT_G,"Dietary Interview - Total Nutrient Intakes, Fi...",2011,2012,Dietary,NaN,2011-2012
5879,DR1TSODI,Sodium (mg),DR1TOT_H,"Dietary Interview - Total Nutrient Intakes, Fi...",2013,2014,Dietary,NaN,2013-2014
6625,DR1TSODI,Sodium (mg),DR1TOT_I,"Dietary Interview - Total Nutrient Intakes, Fi...",2015,2016,Dietary,NaN,2015-2016
7367,DR1TSODI,Sodium (mg),DR1TOT_J,"Dietary Interview - Total Nutrient Intakes, Fi...",2017,2018,Dietary,NaN,2017-2018
8477,DR1TSODI,Sodium (mg),P_DR1TOT,"Dietary Interview - Total Nutrient Intakes, Fi...",2017,2020,Dietary,NaN,2017-2020


2. Changes in same quesion format:
    - `MCQ250A`, `MCQ300A`, and `MCQ300c` are the same question with a change in the formulatting over the years.
    - `LBDHDDSI` and `LBDHDLSI` are the same variable for HDL-choresterol

In [534]:
glance_var_docs("MCQ250A")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
30290,MCQ250A,"Including living and deceased, were any of {SP...",MCQ,Medical Conditions,1999,2000,Questionnaire,NaN,1999-2000
33832,MCQ250A,"Including living and deceased, were any of {SP...",MCQ_B,Medical Conditions,2001,2002,Questionnaire,NaN,2001-2002
32697,MCQ250A,"Including living and deceased, were any of {SP...",MCQ_C,Medical Conditions,2003,2004,Questionnaire,NaN,2003-2004


In [533]:
glance_var_docs("MCQ300C")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
31603,MCQ300C,"Including living and deceased, were any of {SP...",MCQ_D,Medical Conditions,2005,2006,Questionnaire,NaN,2005-2006
28503,MCQ300C,"Including living and deceased, were any of {SP...",MCQ_E,Medical Conditions,2007,2008,Questionnaire,NaN,2007-2008
35450,MCQ300C,"Including living and deceased, were any of {SP...",MCQ_F,Medical Conditions,2009,2010,Questionnaire,NaN,2009-2010


In [535]:
glance_var_docs("MCQ300c")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
40662,MCQ300c,"Including living and deceased, were any of {SP...",MCQ_G,Medical Conditions,2011,2012,Questionnaire,NaN,2011-2012
41620,MCQ300c,"Including living and deceased, were any of {SP...",MCQ_H,Medical Conditions,2013,2014,Questionnaire,NaN,2013-2014
43069,MCQ300c,"Including living and deceased, were any of {SP...",MCQ_I,Medical Conditions,2015,2016,Questionnaire,NaN,2015-2016
44380,MCQ300c,"Including living and deceased, were any of {SP...",MCQ_J,Medical Conditions,2017,2018,Questionnaire,NaN,2017-2018
45817,MCQ300c,"Including living and deceased, were any of {SP...",P_MCQ,Medical Conditions,2017,2020,Questionnaire,NaN,2017-2020


In [572]:
glance_var_docs("LBDHDLSI")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
47584,LBDHDLSI,HDL-cholesterol (mmol/L),Lab13,Cholesterol - Total & HDL,1999,2000,Laboratory,NaN,1999-2000
47967,LBDHDLSI,HDL-cholesterol (mmol/L),l13_b,Cholesterol - Total & HDL,2001,2002,Laboratory,NaN,2001-2002


In [573]:
glance_var_docs("LBDHDDSI")

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints,Year
47971,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),l13_c,Cholesterol - Total & HDL,2003,2004,Laboratory,NaN,2003-2004
47980,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_D,Cholesterol - HDL,2005,2006,Laboratory,NaN,2005-2006
47263,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_E,Cholesterol - HDL,2007,2008,Laboratory,NaN,2007-2008
50841,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_F,Cholesterol - HDL,2009,2010,Laboratory,NaN,2009-2010
52819,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_G,Cholesterol - HDL,2011,2012,Laboratory,NaN,2011-2012
53631,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_H,Cholesterol - HDL,2013,2014,Laboratory,NaN,2013-2014
56277,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_I,Cholesterol - High-Density Lipoprotein (HDL),2015,2016,Laboratory,NaN,2015-2016
58730,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),HDL_J,Cholesterol - High - Density Lipoprotein (HDL),2017,2018,Laboratory,NaN,2017-2018
59824,LBDHDDSI,Direct HDL-Cholesterol (mmol/L),P_HDL,Cholesterol - High - Density Lipoprotein (HDL),2017,2020,Laboratory,NaN,2017-2020
